шифрование

---



In [ ]:
import os

# Константы для алгоритма шифрования Кузнечик
SBOX = [
    252, 238, 221, 17, 207, 110, 49, 22, 251, 196, 250, 218, 35, 197, 4, 77,
    233, 119, 240, 219, 147, 46, 153, 186, 23, 54, 241, 187, 20, 205, 95, 193,
    249, 24, 101, 90, 226, 92, 239, 33, 129, 28, 60, 66, 139, 1, 142, 79,
    5, 132, 2, 174, 227, 106, 143, 160, 6, 11, 237, 152, 127, 212, 211, 31,
    235, 52, 44, 81, 234, 200, 72, 171, 242, 42, 104, 162, 253, 58, 206, 204,
    181, 112, 14, 86, 8, 12, 118, 18, 191, 114, 19, 71, 156, 183, 93, 135,
    21, 161, 150, 41, 16, 123, 154, 199, 243, 145, 120, 111, 157, 158, 178, 177,
    50, 117, 25, 61, 255, 53, 138, 126, 109, 84, 198, 128, 195, 189, 13, 87,
    223, 245, 36, 169, 62, 168, 67, 201, 215, 121, 214, 246, 124, 34, 185, 3,
    224, 15, 236, 222, 122, 148, 176, 188, 220, 232, 40, 80, 78, 51, 10, 74,
    167, 151, 96, 115, 30, 0, 98, 68, 26, 184, 56, 130, 100, 159, 38, 65,
    173, 69, 70, 146, 39, 94, 85, 47, 140, 163, 165, 125, 105, 213, 149, 59,
    7, 88, 179, 64, 134, 172, 29, 247, 48, 55, 107, 228, 136, 217, 231, 137,
    225, 27, 131, 73, 76, 63, 248, 254, 141, 83, 170, 144, 202, 216, 133, 97,
    32, 113, 103, 164, 45, 43, 9, 91, 203, 155, 37, 208, 190, 229, 108, 82,
    89, 166, 116, 210, 230, 244, 180, 192, 209, 102, 175, 194, 57, 75, 99, 182
]

LINEAR_KOEF = [148, 32, 133, 16, 194, 192, 1, 251, 1, 192, 194, 16, 133, 32, 148, 1]

def s_transform(block):
    return bytearray(SBOX[b] for b in block)

def l_transform(block):
    result = bytearray(block)
    for _ in range(16):
        x = 0
        for i in range(16):
            x ^= galois_mul(result[i], LINEAR_KOEF[i])
        result = result[1:] + bytearray([x])
    return result

def galois_mul(a, b):
    p = 0
    for i in range(8):
        if b & 1:
            p ^= a
        hi_bit_set = a & 0x80
        a <<= 1
        if hi_bit_set:
            a ^= 0xc3
        b >>= 1
    return p & 0xff

def expand_key(key):
    keys = [key[:16], key[16:]]
    for i in range(4):
        for j in range(8):
            k = keys[-2] + keys[-1]
            k = l_transform(s_transform(k))
            c = bytearray([0] * 15 + [i * 8 + j + 1])
            k = bytearray(x ^ y for x, y in zip(k, l_transform(c)))
            keys.append(k)
    return keys[:10]

def xor_blocks(block1, block2):
    return bytearray(x ^ y for x, y in zip(block1, block2))

def encrypt_block(block, round_keys):
    state = bytearray(block)
    for i in range(9):
        state = xor_blocks(state, round_keys[i])
        state = s_transform(state)
        state = l_transform(state)
    return xor_blocks(state, round_keys[9])

def encrypt_ecb(data, key):
    round_keys = expand_key(key)
    result = bytearray()
    padding_len = 16 - (len(data) % 16)
    data = bytearray(data) + bytearray([padding_len] * padding_len)
    for i in range(0, len(data), 16):
        block = data[i:i+16]
        encrypted_block = encrypt_block(block, round_keys)
        result.extend(encrypted_block)
    return bytes(result)

def encrypt_file(input_file, output_file, key):
    with open(input_file, 'rb') as f:
        data = f.read()
    encrypted_data = encrypt_ecb(data, key)
    with open(output_file, 'wb') as f:
        f.write(encrypted_data)

if __name__ == "__main__":
    key = os.urandom(32)  # Генерация случайного 256-битного ключа

    input_file = input("Введите имя входного файла для шифрования: ")
    output_file = input("Введите имя выходного файла для зашифрованных данных: ")

    encrypt_file(input_file, output_file, key)
    print(f"Файл успешно зашифрован и сохранен как {output_file}")
    print("Использованный ключ (в шестнадцатеричном виде):")
    print(key.hex())


Расшифрование

In [ ]:
import os

# Константы для алгоритма шифрования Кузнечик
SBOX = [
    252, 238, 221, 17, 207, 110, 49, 22, 251, 196, 250, 218, 35, 197, 4, 77,
    233, 119, 240, 219, 147, 46, 153, 186, 23, 54, 241, 187, 20, 205, 95, 193,
    249, 24, 101, 90, 226, 92, 239, 33, 129, 28, 60, 66, 139, 1, 142, 79,
    5, 132, 2, 174, 227, 106, 143, 160, 6, 11, 237, 152, 127, 212, 211, 31,
    235, 52, 44, 81, 234, 200, 72, 171, 242, 42, 104, 162, 253, 58, 206, 204,
    181, 112, 14, 86, 8, 12, 118, 18, 191, 114, 19, 71, 156, 183, 93, 135,
    21, 161, 150, 41, 16, 123, 154, 199, 243, 145, 120, 111, 157, 158, 178, 177,
    50, 117, 25, 61, 255, 53, 138, 126, 109, 84, 198, 128, 195, 189, 13, 87,
    223, 245, 36, 169, 62, 168, 67, 201, 215, 121, 214, 246, 124, 34, 185, 3,
    224, 15, 236, 222, 122, 148, 176, 188, 220, 232, 40, 80, 78, 51, 10, 74,
    167, 151, 96, 115, 30, 0, 98, 68, 26, 184, 56, 130, 100, 159, 38, 65,
    173, 69, 70, 146, 39, 94, 85, 47, 140, 163, 165, 125, 105, 213, 149, 59,
    7, 88, 179, 64, 134, 172, 29, 247, 48, 55, 107, 228, 136, 217, 231, 137,
    225, 27, 131, 73, 76, 63, 248, 254, 141, 83, 170, 144, 202, 216, 133, 97,
    32, 113, 103, 164, 45, 43, 9, 91, 203, 155, 37, 208, 190, 229, 108, 82,
    89, 166, 116, 210, 230, 244, 180, 192, 209, 102, 175, 194, 57, 75, 99, 182
]

INV_SBOX = [SBOX.index(i) for i in range(256)]

LINEAR_KOEF = [148, 32, 133, 16, 194, 192, 1, 251, 1, 192, 194, 16, 133, 32, 148, 1]

def s_transform(block):
    return bytearray(SBOX[b] for b in block)

def inv_s_transform(block):
    return bytearray(INV_SBOX[b] for b in block)

def galois_mul(a, b):
    p = 0
    for i in range(8):
        if b & 1:
            p ^= a
        hi_bit_set = a & 0x80
        a <<= 1
        if hi_bit_set:
            a ^= 0xc3
        b >>= 1
    return p & 0xff

def l_transform(block):
    result = bytearray(block)
    for _ in range(16):
        x = 0
        for i in range(16):
            x ^= galois_mul(result[i], LINEAR_KOEF[i])
        result = result[1:] + bytearray([x])
    return result

def inv_l_transform(block):
    result = bytearray(block)
    for _ in range(16):
        x = result[15]
        result = bytearray([x]) + result[:15]
        for i in range(16):
            x ^= galois_mul(result[i], LINEAR_KOEF[15 - i])
        result[0] = x
    return result

def expand_key(key):
    keys = [key[:16], key[16:]]
    for i in range(4):
        for j in range(8):
            k = keys[-2] + keys[-1]
            k = l_transform(s_transform(k))
            c = bytearray([0] * 15 + [i * 8 + j + 1])
            k = bytearray(x ^ y for x, y in zip(k, l_transform(c)))
            keys.append(k)
    return keys[:10]

def xor_blocks(block1, block2):
    return bytearray(x ^ y for x, y in zip(block1, block2))

def decrypt_block(block, round_keys):
    state = bytearray(block)
    for i in range(9, 0, -1):
        state = xor_blocks(state, round_keys[i])
        state = inv_l_transform(state)
        state = inv_s_transform(state)
    return xor_blocks(state, round_keys[0])

def decrypt_ecb(data, key):
    round_keys = expand_key(key)
    result = bytearray()
    for i in range(0, len(data), 16):
        block = data[i:i+16]
        decrypted_block = decrypt_block(block, round_keys)
        result.extend(decrypted_block)
    padding_len = result[-1]
    return bytes(result[:-padding_len])

def decrypt_file(input_file, output_file, key):
    with open(input_file, 'rb') as f:
        data = f.read()
    decrypted_data = decrypt_ecb(data, key)
    with open(output_file, 'wb') as f:
        f.write(decrypted_data)

if __name__ == "__main__":
    key_hex = input("Введите ключ в шестнадцатеричном виде: ")
    key = bytes.fromhex(key_hex)

    input_file = input("Введите имя входного файла для расшифрования: ")
    output_file = input("Введите имя выходного файла для расшифрованных данных: ")

    decrypt_file(input_file, output_file, key)
    print(f"Файл успешно расшифрован и сохранен как {output_file}")
